In [1]:
from propositional_state_logic import *
from utils import *
from collections import defaultdict
import itertools
from copy import deepcopy

In [2]:
p = Problem()

# Define the variables above. Returns a Variable object.
F1 = p.add_variable('F1', type='binary')
P1 = p.add_variable('P1', type='binary')
V1 = p.add_variable('V1', type='binary')
P2 = p.add_variable('P2', type='binary')
V2 = p.add_variable('V2', type='binary')
PV2 = p.add_variable('PV2', type='binary')
V3 = p.add_variable('V3', type='binary')
PV3 = p.add_variable('PV3', type='binary')
P3 = p.add_variable('P3', type='binary')
R1 = p.add_variable('R1', type='binary')
T1 = p.add_variable('T1', type='binary')

# Add the word problem constraints.
p.add_constraint('F1 => P1')
p.add_constraint('~F1 => ~P1')

p.add_constraint('V1 & P1 => P2')
p.add_constraint('V1 & ~P1 => ~P2')
p.add_constraint('~V1 => ~P2')

p.add_constraint('V2 & P2 => PV2')
p.add_constraint('V2 & ~P2 => ~PV2')
p.add_constraint('~V2 => ~PV2')

p.add_constraint('V3 & P2 => PV3')
p.add_constraint('V3 & ~P2 => ~PV3')
p.add_constraint('~V3 => ~PV3')

p.add_constraint('~PV2 & ~PV3 => ~P3')
p.add_constraint('PV2 => P3')
p.add_constraint('PV3 => P3')

p.add_constraint('R1 & P3 => T1')
p.add_constraint('R1 & ~P3 => ~T1')
p.add_constraint('~R1 => ~T1')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [3]:
sat = SATSolver(p)
# Make an assignment to some variables; variable.get_assignment(value) returns an Assignment.
assignment = frozenset([F1.get_assignment('True'), P1.get_assignment('False')])
sat.check_consistency(assignment)

(False, None, frozenset())

In [4]:
Vs = ['T1', 'R1', 'P3']

In [5]:
Vs

['T1', 'R1', 'P3']

In [6]:
def findsubsets(s, n):
    return (list(tup) for tup in itertools.combinations(s, n))

In [7]:
nodes = []

for i in range(len(Vs)+1):
    for n in findsubsets(Vs, i):
        nodes.append(n)

print(nodes)

[[], ['T1'], ['R1'], ['P3'], ['T1', 'R1'], ['T1', 'P3'], ['R1', 'P3'], ['T1', 'R1', 'P3']]


In [8]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)

    a = []
    a.append(tuple(n))
    
    if new_nodes == [] or [j for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1] == []:
        a.append([])
        
    else:
        a.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1])
    
    return a

In [9]:
def list_tree(nodes, n):
    new_nodes = deepcopy(nodes)
    if n in new_nodes:
        new_nodes.remove(n)
    
    if new_nodes == [] or [j for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1] == []:
        n.append([])
        
    else:
        n.append([list_tree([k for k in new_nodes if k != j], j) for j in new_nodes if n != j and n == j[:len(n)] and len(j) == len(n)+1])
    
    return n

In [10]:
tree = list_tree(nodes, [])
print(tree)

[[['T1', [['T1', 'R1', [['T1', 'R1', 'P3', []]]], ['T1', 'P3', []]]], ['R1', [['R1', 'P3', []]]], ['P3', []]]]


In [11]:
product = list((list(tup) for tup in itertools.product(['True', 'False'], repeat=len(Vs))))
product

[['True', 'True', 'True'],
 ['True', 'True', 'False'],
 ['True', 'False', 'True'],
 ['True', 'False', 'False'],
 ['False', 'True', 'True'],
 ['False', 'True', 'False'],
 ['False', 'False', 'True'],
 ['False', 'False', 'False']]

In [12]:
all_assignments = []

for prod in product:
    all_assignments.append([p.get_variable(Vs[i]).get_assignment(prod[i]) for i in range(len(Vs))])

all_assignments

[[T1, R1, P3],
 [T1, R1, ¬P3],
 [T1, ¬R1, P3],
 [T1, ¬R1, ¬P3],
 [¬T1, R1, P3],
 [¬T1, R1, ¬P3],
 [¬T1, ¬R1, P3],
 [¬T1, ¬R1, ¬P3]]

In [15]:
def go_through_tree(tree_list):
    if len(tree_list) == 0:
        print(tree_list)
    else:
        return go_through_tree(tree_list[0])

In [16]:
go_through_tree(tree)

RecursionError: maximum recursion depth exceeded while calling a Python object